In [34]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from datetime import datetime
import holidays
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('C:/Users/anbuv/Downloads/archive/Marketing.csv')

In [3]:
df.head()

,id,c_date,campaign_name,category,campaign_id,impressions,mark_spent,clicks,leads,orders,revenue
0,1,2021-02-01,facebook_tier1,social,349043,148263,7307.37,1210,13,1,4981.0
1,2,2021-02-01,facebOOK_tier2,social,348934,220688,16300.20,1640,48,3,14962.0
2,3,2021-02-01,google_hot,search,89459845,22850,5221.60,457,9,1,7981.0
3,4,2021-02-01,google_wide,search,127823,147038,6037.00,1196,24,1,2114.0
4,5,2021-02-01,youtube_blogger,influencer,10934,225800,29962.20,2258,49,10,84490.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             308 non-null    int64  
 1   c_date         308 non-null    object 
 2   campaign_name  308 non-null    object 
 3   category       308 non-null    object 
 4   campaign_id    308 non-null    int64  
 5   impressions    308 non-null    int64  
 6   mark_spent     308 non-null    float64
 7   clicks         308 non-null    int64  
 8   leads          308 non-null    int64  
 9   orders         308 non-null    int64  
 10  revenue        308 non-null    float64
dtypes: float64(2), int64(6), object(3)
memory usage: 26.6+ KB


### Data Cleaning

### Changing the campaign date to datetime type

In [5]:
df['c_date'] = pd.to_datetime(df['c_date'])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             308 non-null    int64         
 1   c_date         308 non-null    datetime64[ns]
 2   campaign_name  308 non-null    object        
 3   category       308 non-null    object        
 4   campaign_id    308 non-null    int64         
 5   impressions    308 non-null    int64         
 6   mark_spent     308 non-null    float64       
 7   clicks         308 non-null    int64         
 8   leads          308 non-null    int64         
 9   orders         308 non-null    int64         
 10  revenue        308 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(6), object(2)
memory usage: 26.6+ KB


In [7]:
start_date = df['c_date'].min()
print('The start date is:', start_date)

The start date is: 2021-02-01 00:00:00


In [8]:
end_date = df['c_date'].max()
print('The end date is:', end_date)

The end date is: 2021-02-28 00:00:00


As we can see the data we have is for a marketing campaign data for the month of February, 2021 for a company X.

### Campaign Name cleaning

In [9]:
print("Unique campaign names:")
print(df['campaign_name'].unique())

Unique campaign names:
['facebook_tier1' 'facebOOK_tier2' 'google_hot' 'google_wide'
 'youtube_blogger' 'instagram_tier1' 'instagram_tier2'
 'facebook_retargeting' 'facebook_lal' 'instagram_blogger'
 'banner_partner']


In [10]:
# Fix the campaign name with incorrect capitalization
df['campaign_name'] = df['campaign_name'].replace('facebOOK_tier2', 'facebook_tier2')

In [11]:
print("Unique campaign names after correction:")
print(df['campaign_name'].unique())

Unique campaign names after correction:
['facebook_tier1' 'facebook_tier2' 'google_hot' 'google_wide'
 'youtube_blogger' 'instagram_tier1' 'instagram_tier2'
 'facebook_retargeting' 'facebook_lal' 'instagram_blogger'
 'banner_partner']


### Data Engineering

There are 11 campaigns in the period on 5 platforms. We will create a new column platform_name which we will use for analysis later.

In [12]:
# Create a function to extract platform name from campaign_name
def get_platform_name(campaign):
    campaign = campaign.lower()
    if 'facebook' in campaign:
        return 'facebook'
    elif 'google' in campaign:
        return 'google' 
    elif 'youtube' in campaign:
        return 'youtube'
    elif 'instagram' in campaign:
        return 'instagram'
    elif 'banner' in campaign:
        return 'banner'
    else:
        return campaign

# Apply the function to create new platform_name column
df['platform_name'] = df['campaign_name'].apply(get_platform_name)

# Display unique platforms to verify
print("Unique platforms after grouping:")
print(df['platform_name'].unique())

Unique platforms after grouping:
['facebook' 'google' 'youtube' 'instagram' 'banner']


In [13]:
# Reorder columns to put platform_name next to campaign_name
cols = ['id', 'c_date', 'campaign_name', 'platform_name', 'category', 'campaign_id', 'impressions', 'mark_spent', 'clicks', 'leads', 'orders', 'revenue']
df = df[cols]
df.head()

,id,c_date,campaign_name,platform_name,category,campaign_id,impressions,mark_spent,clicks,leads,orders,revenue
0,1,2021-02-01,facebook_tier1,facebook,social,349043,148263,7307.37,1210,13,1,4981.0
1,2,2021-02-01,facebook_tier2,facebook,social,348934,220688,16300.20,1640,48,3,14962.0
2,3,2021-02-01,google_hot,google,search,89459845,22850,5221.60,457,9,1,7981.0
3,4,2021-02-01,google_wide,google,search,127823,147038,6037.00,1196,24,1,2114.0
4,5,2021-02-01,youtube_blogger,youtube,influencer,10934,225800,29962.20,2258,49,10,84490.0


In [14]:
# Create day_of_week column using c_date
df['day_of_week'] = pd.to_datetime(df['c_date']).dt.day_name()

# Create is_holiday column
us_holidays = holidays.US()
df['is_holiday'] = df['c_date'].apply(lambda x: 'yes' if x in us_holidays else 'no')


In [15]:
df.head()

,id,c_date,campaign_name,platform_name,category,campaign_id,impressions,mark_spent,clicks,leads,orders,revenue,day_of_week,is_holiday
0,1,2021-02-01,facebook_tier1,facebook,social,349043,148263,7307.37,1210,13,1,4981.0,Monday,no
1,2,2021-02-01,facebook_tier2,facebook,social,348934,220688,16300.20,1640,48,3,14962.0,Monday,no
2,3,2021-02-01,google_hot,google,search,89459845,22850,5221.60,457,9,1,7981.0,Monday,no
3,4,2021-02-01,google_wide,google,search,127823,147038,6037.00,1196,24,1,2114.0,Monday,no
4,5,2021-02-01,youtube_blogger,youtube,influencer,10934,225800,29962.20,2258,49,10,84490.0,Monday,no


### Exploratory Data Analysis

In [16]:
# Basic statistics for numerical columns
print("Basic statistics for numerical columns:")
print(df.describe())

Basic statistics for numerical columns:
               id               c_date   campaign_id   impressions  \
count  308.000000                  308  3.080000e+02  3.080000e+02   
mean   154.500000  2021-02-14 12:00:00  9.605628e+06  5.122475e+06   
min      1.000000  2021-02-01 00:00:00  1.093400e+04  6.670000e+02   
25%     77.750000  2021-02-07 18:00:00  1.278230e+05  1.479568e+05   
50%    154.500000  2021-02-14 12:00:00  3.747540e+05  6.028100e+05   
75%    231.250000  2021-02-21 06:00:00  4.387490e+06  3.254450e+06   
max    308.000000  2021-02-28 00:00:00  8.945984e+07  4.199700e+08   
std     89.056162                  NaN  2.543040e+07  2.631134e+07   

          mark_spent        clicks        leads      orders       revenue  
count     308.000000    308.000000   308.000000  308.000000  3.080000e+02  
mean    99321.038377   9739.996753   212.918831   26.113636  1.392512e+05  
min       169.750000     20.000000     0.000000    0.000000  0.000000e+00  
25%      9465.195000   13

In [17]:
# Check for missing values
print("Missing values in each column:")
print(df.isnull().sum())

Missing values in each column:
id               0
c_date           0
campaign_name    0
platform_name    0
category         0
campaign_id      0
impressions      0
mark_spent       0
clicks           0
leads            0
orders           0
revenue          0
day_of_week      0
is_holiday       0
dtype: int64


In [18]:
# Count zeros in each column
print("Number of zeros in each column:")
zero_counts = (df == 0).sum()
print(zero_counts)

# Count campaigns with zero leads or orders
zero_leads = len(df[df['leads'] == 0])
zero_orders = len(df[df['orders'] == 0])

print(f"\nNumber of campaigns with:")
print(f"Zero leads: {zero_leads}")
print(f"Zero orders: {zero_orders}")


Number of zeros in each column:
id                0
c_date            0
campaign_name     0
platform_name     0
category          0
campaign_id       0
impressions       0
mark_spent        0
clicks            0
leads             4
orders           19
revenue          19
day_of_week       0
is_holiday        0
dtype: int64

Number of campaigns with:
Zero leads: 4
Zero orders: 19


As we can see there are 19 records where we have zero orders and revenue. We also have 4 records with zero leads

In [20]:
# Filter records where both leads and orders are zero
zero_leads_orders = df[(df['leads'] == 0) | (df['orders'] == 0)]

# Display relevant columns
print("Campaigns with zero leads and orders:")
print(zero_leads_orders[['c_date', 'campaign_name', 'leads', 'orders', 'revenue', 'day_of_week', 'is_holiday']])


Campaigns with zero leads and orders:
        c_date         campaign_name  leads  orders  revenue day_of_week  \
10  2021-02-01        banner_partner      3       0      0.0      Monday   
12  2021-02-02        facebook_tier2     10       0      0.0     Tuesday   
17  2021-02-02       instagram_tier2     14       0      0.0     Tuesday   
18  2021-02-02  facebook_retargeting      0       0      0.0     Tuesday   
82  2021-02-08       instagram_tier1      1       0      0.0      Monday   
115 2021-02-11       instagram_tier1      1       0      0.0    Thursday   
247 2021-02-23       instagram_tier1      1       0      0.0     Tuesday   
266 2021-02-25            google_hot      1       0      0.0    Thursday   
277 2021-02-26            google_hot      2       0      0.0      Friday   
283 2021-02-26          facebook_lal      0       0      0.0      Friday   
287 2021-02-27        facebook_tier2     13       0      0.0    Saturday   
288 2021-02-27            google_hot      2       

As we can see from the above step, zero order days are spread out through out the week. 

### Marketing Metrics:
* conversion_rate: Percentage of clicks that result in orders (orders/clicks * 100) ----- Shows how effectively we convert visitors into customers
* cost_per_click (CPC): Marketing spend divided by number of clicks (mark_spent/clicks) ---- Shows how much we pay for each click/visitor
* cost_per_lead (CPL): Marketing spend divided by number of leads (mark_spent/leads) ---- Shows how much we pay to acquire each lead
* cost_per_order (CPO): Marketing spend divided by number of orders (mark_spent/orders) ---- Shows how much we pay to acquire each paying customer
* return on ad spend (ROAS): Revenue divided by marketing spend (revenue/mark_spent) ---- Shows how much revenue we generate for each dollar spent on marketing
* Click-through rate (CTR)
* Return on Investment (ROI)

In [22]:
# Calculate key marketing metrics with handling for zero values
df['conversion_rate'] = (df['orders'] / df['clicks']).replace([np.inf, -np.inf], 0) * 100
df['cost_per_click'] = (df['mark_spent'] / df['clicks']).replace([np.inf, -np.inf], 0)
df['cost_per_lead'] = (df['mark_spent'] / df['leads']).replace([np.inf, -np.inf], 0)
df['cost_per_order'] = (df['mark_spent'] / df['orders']).replace([np.inf, -np.inf], 0)
df['roas'] = (df['revenue'] / df['mark_spent']).replace([np.inf, -np.inf], 0)
df['ctr'] = (df['clicks'] / df['impressions'] * 100).replace([np.inf,-np.inf],0)
df['roi'] = (df['revenue'] - df['mark_spent']/df['mark_spent']*100).replace([np.inf,-np.inf],0)


In [24]:
df.head().T

,0,1,2,3,4
id,1,2,3,4,5
c_date,2021-02-01 00:00:00,2021-02-01 00:00:00,2021-02-01 00:00:00,2021-02-01 00:00:00,2021-02-01 00:00:00
campaign_name,facebook_tier1,facebook_tier2,google_hot,google_wide,youtube_blogger
platform_name,facebook,facebook,google,google,youtube
category,social,social,search,search,influencer
campaign_id,349043,348934,89459845,127823,10934
impressions,148263,220688,22850,147038,225800
mark_spent,7307.37,16300.2,5221.6,6037.0,29962.2
clicks,1210,1640,457,1196,2258
leads,13,48,9,24,49


In [27]:
# Calculate metrics by category
category_metrics = df.groupby('category').agg({
    'orders': 'sum',
    'clicks': 'sum',
    'mark_spent': 'sum',
    'leads': 'sum', 
    'revenue': 'sum',
    'impressions': 'sum'
}).assign(
    conversion_rate=lambda x: round((x['orders'] / x['clicks'] * 100).replace([np.inf, -np.inf], 0), 2),
    cost_per_click=lambda x: round((x['mark_spent'] / x['clicks']).replace([np.inf, -np.inf], 0), 2),
    cost_per_lead=lambda x: round((x['mark_spent'] / x['leads']).replace([np.inf, -np.inf], 0), 2),
    cost_per_order=lambda x: round((x['mark_spent'] / x['orders']).replace([np.inf, -np.inf], 0), 2),
    roas=lambda x: round((x['revenue'] / x['mark_spent']).replace([np.inf, -np.inf], 0), 2),
    ctr=lambda x: round((x['clicks'] / x['impressions'] * 100).replace([np.inf, -np.inf], 0), 2),
    roi=lambda x: round(((x['revenue'] - x['mark_spent']) / x['mark_spent'] * 100).replace([np.inf, -np.inf], 0), 2)
)

print("\nMetrics by Category:")
print(category_metrics[['conversion_rate', 'cost_per_click', 'cost_per_lead', 
                       'cost_per_order', 'roas', 'ctr', 'roi']])

# Calculate metrics by platform
platform_metrics = df.groupby('platform_name').agg({
    'orders': 'sum',
    'clicks': 'sum',
    'mark_spent': 'sum',
    'leads': 'sum',
    'revenue': 'sum',
    'impressions': 'sum'
}).assign(
    conversion_rate=lambda x: round((x['orders'] / x['clicks'] * 100).replace([np.inf, -np.inf], 0), 2),
    cost_per_click=lambda x: round((x['mark_spent'] / x['clicks']).replace([np.inf, -np.inf], 0), 2),
    cost_per_lead=lambda x: round((x['mark_spent'] / x['leads']).replace([np.inf, -np.inf], 0), 2),
    cost_per_order=lambda x: round((x['mark_spent'] / x['orders']).replace([np.inf, -np.inf], 0), 2),
    roas=lambda x: round((x['revenue'] / x['mark_spent']).replace([np.inf, -np.inf], 0), 2),
    ctr=lambda x: round((x['clicks'] / x['impressions'] * 100).replace([np.inf, -np.inf], 0), 2),
    roi=lambda x: round(((x['revenue'] - x['mark_spent']) / x['mark_spent'] * 100).replace([np.inf, -np.inf], 0), 2)
)

print("\nMetrics by Platform:")
print(platform_metrics[['conversion_rate', 'cost_per_click', 'cost_per_lead',
                       'cost_per_order', 'roas', 'ctr', 'roi']])

# Calculate metrics by campaign
campaign_metrics = df.groupby('campaign_name').agg({
    'orders': 'sum', 
    'clicks': 'sum',
    'mark_spent': 'sum',
    'leads': 'sum',
    'revenue': 'sum',
    'impressions': 'sum'
}).assign(
    conversion_rate=lambda x: round((x['orders'] / x['clicks'] * 100).replace([np.inf, -np.inf], 0), 2),
    cost_per_click=lambda x: round((x['mark_spent'] / x['clicks']).replace([np.inf, -np.inf], 0), 2),
    cost_per_lead=lambda x: round((x['mark_spent'] / x['leads']).replace([np.inf, -np.inf], 0), 2),
    cost_per_order=lambda x: round((x['mark_spent'] / x['orders']).replace([np.inf, -np.inf], 0), 2),
    roas=lambda x: round((x['revenue'] / x['mark_spent']).replace([np.inf, -np.inf], 0), 2),
    ctr=lambda x: round((x['clicks'] / x['impressions'] * 100).replace([np.inf, -np.inf], 0), 2),
    roi=lambda x: round(((x['revenue'] - x['mark_spent']) / x['mark_spent'] * 100).replace([np.inf, -np.inf], 0), 2)
)

print("\nMetrics by Campaign:")
print(campaign_metrics[['conversion_rate', 'cost_per_click', 'cost_per_lead',
                       'cost_per_order', 'roas', 'ctr', 'roi']])



Metrics by Category:
            conversion_rate  cost_per_click  cost_per_lead  cost_per_order  \
category                                                                     
influencer             0.40           11.07         490.31         2755.58   
media                  0.37           11.97         495.29         3209.88   
search                 0.25           10.48         486.90         4179.23   
social                 0.18            9.20         439.67         5236.62   

            roas   ctr     roi  
category                        
influencer  2.54  0.96  154.29  
media       1.22  0.04   22.41  
search      1.07  0.46    7.07  
social      0.86  0.42  -13.68  

Metrics by Platform:
               conversion_rate  cost_per_click  cost_per_lead  cost_per_order  \
platform_name                                                                   
banner                    0.37           11.97         495.29         3209.88   
facebook                  0.22           14.12

In [35]:
# Average Marketing Metric
print("Average marketing metrics:")
metrics = ['conversion_rate', 'cost_per_click', 'cost_per_lead', 'cost_per_order', 'roas']
print(df[metrics].mean())

Average marketing metrics:
conversion_rate       0.279176
cost_per_click       11.270120
cost_per_lead       540.941018
cost_per_order     3960.360669
roas                  1.407612
dtype: float64


In [33]:
# Platform performance comparison
print("Platform-wise performance metrics:")
platform_metrics = df.groupby('platform_name')[['impressions', 'clicks', 'leads', 'orders', 'revenue', 'mark_spent']].sum()
print(platform_metrics)

Platform-wise performance metrics:
               impressions   clicks  leads  orders     revenue   mark_spent
platform_name                                                              
banner          1068337427   420003  10149    1566   6152960.0   5026674.76
facebook         146342429   719936  14378    1564   6696870.0  10167069.91
google            72298252   330054   7107     828   3705065.0   3460400.07
instagram        247055978  1079912  24014    2171  11023038.0   7878798.57
youtube           43688313   450014   9931    1914  15311433.0   4057936.51


In [29]:
print("Average Platform-wise performance metrics:")
platform_metrics_avg = df.groupby('platform_name')[['impressions', 'clicks', 'leads', 'orders', 'revenue', 'mark_spent']].mean()
print(platform_metrics_avg)

Average Platform-wise performance metrics:
                impressions        clicks       leads     orders  \
platform_name                                                      
banner         3.815491e+07  15000.107143  362.464286  55.928571   
facebook       1.306629e+06   6428.000000  128.375000  13.964286   
google         1.291040e+06   5893.821429  126.910714  14.785714   
instagram      2.941143e+06  12856.095238  285.880952  25.845238   
youtube        1.560297e+06  16071.928571  354.678571  68.357143   

                     revenue     mark_spent  
platform_name                                
banner         219748.571429  179524.098571  
facebook        59793.482143   90777.409911  
google          66161.875000   61792.858393  
instagram      131226.642857   93795.221071  
youtube        546836.892857  144926.303929  


In [18]:
# Category-wise analysis
print("Category-wise performance:")
print(df.groupby('category')[['revenue', 'mark_spent', 'orders']].sum())

Category-wise performance:
               revenue   mark_spent  orders
category                                   
influencer  21119887.0   8305304.08    3014
media        6152960.0   5026674.76    1566
search       3705065.0   3460400.07     828
social      11911454.0  13798500.91    2635


In [19]:
print("Category-wise average performance:")
print(df.groupby('category')[['revenue', 'mark_spent', 'orders']].mean())

Category-wise average performance:
                  revenue     mark_spent     orders
category                                           
influencer  377140.839286  148309.001429  53.821429
media       219748.571429  179524.098571  55.928571
search       66161.875000   61792.858393  14.785714
social       70901.511905   82133.933988  15.684524


In [ ]:
# Daily trends
print("Daily performance metrics:")
daily_metrics = df.groupby('c_date')[['impressions', 'clicks', 'revenue']].sum()
print(daily_metrics)

Daily performance metrics:
            impressions  clicks    revenue
c_date                                    
2021-02-01     22803592   14991   182676.0
2021-02-02     46221902   29977   316369.0
2021-02-03    140541998   89972  1583142.0
2021-02-04     58853025  119976  1633899.0
2021-02-05     42506259  150009  2704723.0
2021-02-06     46269355   29996   416199.0
2021-02-07      9269431   29987   434647.0
2021-02-08      6385079   29952   308655.0
2021-02-09      8113096   29977   437923.0
2021-02-10     16137532   59994   811363.0
2021-02-11     19919970   89988  1563209.0
2021-02-12     30842580  120019  1657464.0
2021-02-13     37516676  150017  2802893.0
2021-02-14     34954508  179977  2197128.0
2021-02-15     35881351  210012  2795172.0
2021-02-16    100483322  239979  3620594.0
2021-02-17     78040007  270005  4139131.0
2021-02-18    472888319  299995  2659560.0
2021-02-19     76466877  330047  4973240.0
2021-02-20     82185688  359986  5261521.0
2021-02-21     46112718   3

In [30]:
# Click-through rate (CTR) by platform
df['ctr'] = (df['clicks'] / df['impressions']) * 100
print("Average CTR by platform:")
print(df.groupby('platform_name')['ctr'].mean())

Average CTR by platform:
platform_name
banner       0.129007
facebook     1.261973
google       1.229689
instagram    0.642654
youtube      0.987335
Name: ctr, dtype: float64


In [24]:
# Top performing campaigns
print("Top 5 campaigns by revenue:")
print(df.nlargest(5, 'revenue')[['campaign_name','c_date', 'revenue', 'mark_spent']])


Top 5 campaigns by revenue:
       campaign_name     c_date    revenue  mark_spent
202  youtube_blogger 2021-02-19  2812520.0    880357.0
213  youtube_blogger 2021-02-20  1452540.0    303860.0
136  youtube_blogger 2021-02-13  1296550.0    205045.0
180  youtube_blogger 2021-02-17  1189170.0    363811.0
48   youtube_blogger 2021-02-05  1165230.0    442617.0


In [25]:
# Campaign efficiency
print("Campaigns sorted by ROAS:")
print(df.sort_values('roas', ascending=False)[['campaign_name', 'roas', 'revenue', 'mark_spent']].head())


10. Campaigns sorted by ROAS:
            campaign_name      roas    revenue  mark_spent
103       youtube_blogger  6.593987   130768.0    19831.40
136       youtube_blogger  6.323246  1296550.0   205045.00
40   facebook_retargeting  5.968953     5287.0      885.75
224       youtube_blogger  5.182299   159620.0    30801.00
158       youtube_blogger  5.163309   782138.0   151480.00


In [31]:
# Lead to order conversion rate by category
df['lead_to_order_rate'] = (df['orders'] / df['leads']) * 100
print("Lead to order conversion rate by category:")
print(df.groupby('category')['lead_to_order_rate'].mean())

Lead to order conversion rate by category:
category
influencer    17.525802
media         14.603305
search        11.482287
social        11.835540
Name: lead_to_order_rate, dtype: float64


In [32]:
# Campaign count by platform
print("Number of campaigns per platform:")
print(df.groupby('platform_name')['campaign_name'].count())

Number of campaigns per platform:
platform_name
banner        28
facebook     112
google        56
instagram     84
youtube       28
Name: campaign_name, dtype: int64


In [28]:
#ROI analysis by category
df['roi'] = ((df['revenue'] - df['mark_spent']) / df['mark_spent']) * 100
print("Average ROI by category:")
print(df.groupby('category')['roi'].mean())


Average ROI by category:
category
influencer    170.003927
media          24.226054
search         19.141372
social          7.642676
Name: roi, dtype: float64


In [34]:
# ROI analysis by platform
df['roi'] = ((df['revenue'] - df['mark_spent']) / df['mark_spent']) * 100
print('Average ROI by platform:')
print(df.groupby('platform_name')['roi'].mean())

Average ROI by platform:
platform_name
banner        24.226054
facebook       5.303523
google        19.141372
instagram     19.259925
youtube      306.870044
Name: roi, dtype: float64


In [37]:
# ROI analysis by campaign
df['roi'] = ((df['revenue'] - df['mark_spent']) / df['mark_spent']) * 100
print('Average ROI by campaign:')
print(df.groupby('campaign_name')['roi'].mean())

Average ROI by campaign:
campaign_name
banner_partner           24.226054
facebook_lal            -88.089815
facebook_retargeting    145.026263
facebook_tier1           -6.637590
facebook_tier2          -29.084767
google_hot               70.865333
google_wide             -32.582589
instagram_blogger        33.137810
instagram_tier1          64.592078
instagram_tier2         -39.950113
youtube_blogger         306.870044
Name: roi, dtype: float64


In [35]:
#Cost per click (CPC) analysis
df['cpc'] = df['mark_spent'] / df['clicks']
print("Average CPC by category:")
print(df.groupby('category')['cpc'].mean())

Average CPC by category:
category
influencer    11.619038
media         10.451950
search        10.878875
social        11.420590
Name: cpc, dtype: float64


In [36]:
#Cost per click (CPC) analysis
df['cpc'] = df['mark_spent'] / df['clicks']
print("Average CPC by platform:")
print(df.groupby('platform_name')['cpc'].mean())

Average CPC by platform:
platform_name
banner       10.451950
facebook     14.690767
google       10.878875
instagram     7.774153
youtube       9.676093
Name: cpc, dtype: float64


In [38]:
#Cost per click (CPC) analysis
df['cpc'] = df['mark_spent'] / df['clicks']
print("Average CPC by campaign:")
print(df.groupby('campaign_name')['cpc'].mean())

Average CPC by campaign:
campaign_name
banner_partner          10.451950
facebook_lal            22.882911
facebook_retargeting     9.011788
facebook_tier1          13.657477
facebook_tier2          13.210893
google_hot              13.193599
google_wide              8.564151
instagram_blogger       13.561984
instagram_tier1          7.882315
instagram_tier2          1.878159
youtube_blogger          9.676093
Name: cpc, dtype: float64


In [41]:
# Cost per lead (CPL) analysis
df['cpl'] = (df['mark_spent'] / df['leads']).replace([np.inf, -np.inf], 0)
print("CPL comparison across categories:") 
print(df.groupby('category')['cpl'].mean())

CPL comparison across categories:
category
influencer    501.980490
media         456.060290
search        534.135126
social        570.343279
Name: cpl, dtype: float64


In [44]:
# Cost per lead (CPL) analysis
df['cpl'] = df['mark_spent'] / df['leads']  
df['cpl'] = df['cpl'].replace([np.inf, -np.inf], 0)  # Handle infinities after calculation
print("CPL comparison across campaigns:")
print(df.groupby('campaign_name')['cpl'].mean())

CPL comparison across campaigns:
campaign_name
banner_partner           456.060290
facebook_lal            1325.114512
facebook_retargeting     408.982956
facebook_tier1           645.935710
facebook_tier2           552.082404
google_hot               668.505406
google_wide              399.764846
instagram_blogger        588.802165
instagram_tier1          396.585072
instagram_tier2           93.359015
youtube_blogger          415.158816
Name: cpl, dtype: float64


In [45]:
# Cost per lead (CPL) analysis
df['cpl'] = df['mark_spent'] / df['leads']  
df['cpl'] = df['cpl'].replace([np.inf, -np.inf], 0)  # Handle infinities after calculation
print("CPL comparison across platforms:")
print(df.groupby('platform_name')['cpl'].mean())

CPL comparison across platforms:
platform_name
banner       456.060290
facebook     733.028896
google       534.135126
instagram    359.582084
youtube      415.158816
Name: cpl, dtype: float64


In [46]:
#Revenue per impression
df['revenue_per_impression'] = df['revenue'] / df['impressions']
print("Revenue per impression by category:")
print(df.groupby('category')['revenue_per_impression'].mean())

Revenue per impression by category:
category
influencer    0.276901
media         0.020043
search        0.233021
social        0.142015
Name: revenue_per_impression, dtype: float64


In [47]:
#Average order value
df['avg_order_value'] = df['revenue'] / df['orders']
print("Average order value by platform:")
print(df.groupby('platform_name')['avg_order_value'].mean())

Average order value by platform:
platform_name
banner       3888.777484
facebook     4003.660317
google       5359.647059
instagram    4402.384615
youtube      8046.609172
Name: avg_order_value, dtype: float64


In [47]:
#Lead conversion rate
df['lead_conversion_rate'] = (df['leads'] / df['clicks']) * 100
print("Lead conversion rate by category:")
print(df.groupby('category')['lead_conversion_rate'].mean())


21. Lead conversion rate by category:
category
influencer    2.309248
media         2.215001
search        2.043556
social        2.017304
Name: lead_conversion_rate, dtype: float64


In [48]:
#Revenue contribution analysis
print("Revenue contribution percentage by platform:")
platform_revenue = df.groupby('platform_name')['revenue'].sum()
print((platform_revenue / df['revenue'].sum()) * 100)



22. Revenue contribution percentage by platform:
platform_name
banner       14.346120
facebook     15.614290
google        8.638656
instagram    25.701098
youtube      35.699835
Name: revenue, dtype: float64


In [49]:
#Marketing efficiency ratio
df['mer'] = df['revenue'] / df['mark_spent']
print("Marketing efficiency ratio by category:")
print(df.groupby('category')['mer'].mean())


23. Marketing efficiency ratio by category:
category
influencer    2.700039
media         1.242261
search        1.191414
social        1.076427
Name: mer, dtype: float64


In [48]:
# Cost structure analysis
print("Marketing spend percentage by platform:")
platform_spend = df.groupby('platform_name')['mark_spent'].sum()
print((platform_spend / df['mark_spent'].sum()) * 100)

Marketing spend percentage by platform:
platform_name
banner       16.431939
facebook     33.235624
google       11.311868
instagram    25.755384
youtube      13.265184
Name: mark_spent, dtype: float64


In [51]:
# Orders per impression
df['orders_per_impression'] = (df['orders'] / df['impressions']) * 1000  # per 1000 impressions
print("\n25. Orders per 1000 impressions by platform:")
print(df.groupby('platform_name')['orders_per_impression'].mean())


25. Orders per 1000 impressions by platform:
platform_name
banner       0.005154
facebook     0.044458
google       0.032725
instagram    0.016270
youtube      0.046269
Name: orders_per_impression, dtype: float64


In [53]:
#Cost per order (CPO)
df['cpo'] = df['mark_spent'] / df['orders']
print("Average cost per order by platform:")
print(df.groupby('platform_name')['cpo'].mean())


27. Average cost per order by platform:
platform_name
banner               inf
facebook             inf
google               inf
instagram            inf
youtube      2090.787332
Name: cpo, dtype: float64


In [54]:
#Impression to lead conversion
df['impression_to_lead'] = (df['leads'] / df['impressions']) * 100
print("\n28. Impression to lead conversion rate by category:")
print(df.groupby('category')['impression_to_lead'].mean())


28. Impression to lead conversion rate by category:
category
influencer    0.023237
media         0.003373
search        0.024639
social        0.019211
Name: impression_to_lead, dtype: float64


In [55]:
#Revenue per click
df['revenue_per_click'] = df['revenue'] / df['clicks']
print("\n29. Average revenue per click by platform:")
print(df.groupby('platform_name')['revenue_per_click'].mean())


29. Average revenue per click by platform:
platform_name
banner       12.893568
facebook     11.540090
google       14.300331
instagram    11.194747
youtube      36.802128
Name: revenue_per_click, dtype: float64


In [56]:
# Campaign efficiency score
# Combining multiple metrics into a single score
df['efficiency_score'] = (df['roas'] * 0.4) + (df['ctr'] * 0.2) + (df['lead_conversion_rate'] * 0.2) + (df['lead_to_order_rate'] * 0.2)
print("\n30. Top campaigns by efficiency score:")
print(df.sort_values('efficiency_score', ascending=False)[['campaign_name', 'efficiency_score']].head())



30. Top campaigns by efficiency score:
            campaign_name  efficiency_score
282  facebook_retargeting          9.529246
249  facebook_retargeting          9.386392
271  facebook_retargeting          9.371584
7    facebook_retargeting          9.146240
227  facebook_retargeting          8.443414


1. Performance by Category: 

Influencer campaigns are the most effective overall:
* Highest conversion rate (40%) and ROI (154.29).
* Best cost efficiency with the lowest cost per lead ($490.31) and cost per order ($2755.58).
* Highest ROAS (2.54), indicating strong returns for every dollar spent.

Social campaigns are underperforming:
* Lowest conversion rate (18%) and negative ROI (-13.68).
* High cost per lead ($439.67) and cost per order ($5236.62), making them inefficient.

Recommendation:
* Focus on scaling influencer campaigns while revisiting or optimizing social campaigns to improve their performance.

2. Performance by Platform
YouTube is the top-performing platform:
* Highest conversion rate (43%), ROAS (3.77), and ROI (277.32).
* Low cost per order ($2120.13) and cost per lead ($408.61), making it highly efficient.

Facebook is struggling:
* Negative ROI (-34.13) despite a decent CTR (0.49).
* High cost per order ($6500.68) and cost per lead ($707.13), making it the least efficient platform.

Instagram shows potential:
* Low cost per click ($7.30) and moderate ROI (39.91), but its conversion rate (20%) could be improved.

Recommendation:
* Invest more in YouTube campaigns, particularly YouTube Blogger campaigns, as they deliver exceptional returns. Reevaluate Facebook campaigns to identify inefficiencies, and consider optimizing Instagram campaigns to increase conversions.

3. Performance by Campaign
Top-performing campaigns:
- YouTube Blogger campaign leads with:
* Highest conversion rate (43%), ROAS (3.77), and ROI (277.32).
* Lowest cost per order ($2120.13).

Facebook Retargeting campaign performs well with:
* High conversion rate (36%) and ROI (101.50).
* Cost efficiency compared to other Facebook campaigns.

Underperforming campaigns:
Facebook LAL campaign has:
* Negative ROI (-88.64) and high costs, making it highly inefficient.
* High cost per lead ($1383.94) and cost per order ($8986.19).

Instagram Tier 2 campaign has:
* Lowest conversion rate (6%) and negative ROI (-37.11).

Recommendation:
* Prioritize YouTube Blogger and Facebook Retargeting campaigns for scaling efforts. Discontinue or significantly optimize Facebook LAL and Instagram Tier 2 campaigns to reduce wasted spend.

4. General Observations
* Influencer marketing consistently outperforms other categories in terms of ROI, ROAS, and conversion rates.
* Platforms like YouTube and Instagram show strong potential for growth due to their combination of high engagement (CTR) and relatively low costs.
* Social campaigns, particularly on Facebook, require closer attention as they are currently delivering poor returns.

Final Recommendations

Scale Investments in Influencer Marketing:
* Focus on influencer-driven campaigns as they deliver the best combination of high conversions, low costs, and strong returns.

Increase Budget for YouTube Campaigns:
* Specifically invest in YouTube Blogger campaigns as they outperform all others across key metrics.

Optimize or Discontinue Underperforming Campaigns:
* Reevaluate Facebook LAL and Instagram Tier 2 campaigns to either improve their efficiency or reallocate budgets to better-performing channels.

Test New Strategies for Social Campaigns:
* Experiment with creative content or audience targeting for social platforms like Facebook to improve their ROI.

Monitor Cost Efficiency Across Platforms:
* Keep an eye on rising costs for platforms like Facebook to ensure they remain within acceptable thresholds relative to performance.